In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_dataset = pd.read_csv('../input/tabular-playground-series-may-2022/train.csv')
print(train_dataset)

In [ ]:
test_dataset = pd.read_csv('../input/tabular-playground-series-may-2022/test.csv')
print(test_dataset)

In [ ]:
train_dataset.info()

In [ ]:
train_dataset.isnull().sum()

In [ ]:
import seaborn as sns
sns.distplot(train_dataset['target'])

In [ ]:
train_dataset['f_27'].value_counts()

In [ ]:
##train_dataset['f_27'] = train_dataset['f_27'].str.strip()

f27_dataset =  train_dataset.f_27.str.split('',expand=True).add_prefix('f_27_')
print(f27_dataset)

In [ ]:
print(train_dataset)

In [ ]:
f27_dataset = f27_dataset.drop(['f_27_0', 'f_27_11'], axis =1)

In [ ]:
f27_dataset.columns

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

le = preprocessing.LabelEncoder()
for col in f27_dataset.columns:
    f27_dataset[col] = le.fit_transform(f27_dataset[col])


##f27_dataset = OneHotEncoder().fit_transform(f27_dataset)
print(f27_dataset)

In [ ]:
train_dataset = pd.concat([train_dataset, f27_dataset], axis=1)
train_dataset = train_dataset.drop(['f_27'], axis =1)

print(train_dataset)

In [ ]:
test_dataset.info()

In [ ]:
test_dataset.isnull().sum()

In [ ]:
f27_test_dataset =  test_dataset.f_27.str.split('',expand=True).add_prefix('f_27_')
print(f27_test_dataset)

In [ ]:
f27_test_dataset = f27_test_dataset.drop(['f_27_0', 'f_27_11'], axis =1)

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

le = preprocessing.LabelEncoder()
for col in f27_test_dataset.columns:
    f27_test_dataset[col] = le.fit_transform(f27_test_dataset[col])


print(f27_test_dataset)

In [ ]:
test_dataset = pd.concat([test_dataset, f27_test_dataset], axis=1)
test_dataset = test_dataset.drop(['f_27'], axis =1)

print(test_dataset)

In [ ]:
y = train_dataset.target
x = train_dataset.drop(['target', 'id'], axis =1)
x_test = test_dataset.drop(['id'], axis =1)

print(x.shape)
print(y.shape)
print(x_test.shape)


In [ ]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
x = scaler.fit_transform(x)
print(x)

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
x_test = scaler.fit_transform(x_test)
print(x_test)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split( x, y, test_size=0.33, random_state=42)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [ ]:
model = Sequential()
model.add(Dense(80, input_dim=40, activation='relu'))
model.add(Dense(40, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    

In [ ]:
model.fit(X_train, y_train, epochs=50)

In [ ]:
scores = model.evaluate(X_train, y_train)

In [ ]:
from sklearn.metrics import auc
from sklearn.metrics import roc_curve

y_valid_predicted = model.predict(X_valid)

false_positive_rate, true_positive_rate, thresholds = roc_curve(y_valid, model.predict(X_valid))
print(auc(false_positive_rate, true_positive_rate))

In [ ]:
scores = model.evaluate(X_valid, y_valid)

In [ ]:
predict_x=model.predict(x_test)


In [ ]:
predict_x

In [ ]:
import itertools
predict_x = list(itertools.chain(*predict_x))

In [ ]:
final_result = pd.DataFrame({'id': test_dataset.id, 'target': predict_x})
print(final_result)

In [ ]:
final_result.to_csv('prop_prediction_file.csv', index=False)